### PACKAGES

In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import numpy as np
from sklearn.utils import shuffle

### DEFINE FUNCTIONS

In [20]:
def timing(f):
    def wrap(*args):
        time1 = time()
        ret = f(*args)
        time2 = time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

In [21]:
@timing
def run_func():
    print('hi')
run_func()

hi
run_func function took 0.995 ms


### IMPORT DATA

In [28]:
@timing
def import_data():
    df = pd.read_csv('..\data\\PENN_SP500_daily.csv')
    #df = shuffle(df, random_state=0)
    df = df[['date',
    'TICKER',
    'COMNAM',
    'BIDLO',
    'ASKHI',
    'PRC',
    'VOL',
    'RET',
    'BID',
    'ASK',
    'SHROUT',
    'OPENPRC']]

    df.head()
    
import_data()

C:\PROGRA1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (5,6,9,19,48,56) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


import_data function took 17288.044 ms


In [30]:
df.columns = ['date', 'symbol', 'name', 'bidlo', 'askhi', 'price', 'vol', 'ret', 'bid', 'ask', 'sout', 'open']
df.head()

,date,symbol,name,bidlo,askhi,price,vol,ret,bid,ask,sout,open
0,20000103,ORCL,ORACLE CORP,111.6250,125.1875,118.1250,24831819.0,0.054099,118.1250,118.1875,1423672.0,124.62500
1,20000104,ORCL,ORACLE CORP,105.0000,118.6250,107.6875,29568565.0,-0.08836,107.6875,107.7500,1423672.0,115.50000
2,20000105,ORCL,ORACLE CORP,96.0000,106.3750,102.0000,42032160.0,-0.052815,102.0000,102.0000,1423672.0,101.62500
3,20000106,ORCL,ORACLE CORP,94.6875,105.0000,96.0000,27848885.0,-0.058824,96.0000,95.8750,1423672.0,100.15625
4,20000107,ORCL,ORACLE CORP,93.5625,103.5000,103.3750,23218034.0,0.076823,103.2500,103.3750,1423672.0,95.00000


### DROP DUPLICATES

In [31]:
list_symbol = df['symbol'].unique()
list_symbol[0]

'ORCL'

### WORKSPACE

In [32]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')

### NAME TABLE

In [33]:
df_name = df[['symbol', 'name']]
df_name = df_name.drop_duplicates()
df_name.index = df_name['symbol']
df_name = df_name.drop('symbol', axis=1)
df_name.head()

,name
symbol,
ORCL,ORACLE CORP
MSFT,MICROSOFT CORP
TROW,T ROWE PRICE ASSOC INC
TROW,T ROWE PRICE GROUP INC
HON,HONEYWELL INTERNATIONAL INC


### BIDLO

In [41]:
@timing
def bidlo_func():

    bidlo_error = []
    df_bidlo = df[['date', 'symbol', 'bidlo']]
    ########----------------#######
    df_bidlo_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:
            df_temp = df_bidlo[df_bidlo['symbol']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['bidlo']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_bidlo_temp = pd.concat([df_bidlo_temp, df_temp], axis=0) ### TEMP
        except:
            bidlo_error.append(symbol)
            
    return df_bidlo_temp, bidlo_error
    
df_bidlo, error_bidlo = bidlo_func()
print(len(error_bidlo))
df_bidlo.head(2)

bidlo_func function took 97652.284 ms


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,111.625,105.0,96.0000,94.6875,93.5625,105.500,109.5000,103.6875,103.50000,104.750,...,45.53,45.44,45.75,45.48,43.86,42.68,42.40,43.46,44.74,44.84
TROW,34.625,34.0,33.1875,33.3750,34.5000,34.875,34.5625,35.1875,36.59375,38.375,...,90.69,90.33,88.07,86.91,86.39,84.79,84.59,88.00,90.68,91.33


In [44]:
len(error_bidlo)

104

In [58]:
pd.Series(error_bidlo).to_csv('error_bidlo.csv')

C:\PROGRA1\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


### BID TABLE

In [46]:
@timing
def bid_func():

    error_bid = []
    df_bid = df[['date', 'symbol', 'bid']]
    df_bid_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:        
            df_temp = df_bid[df_bid['symbol']==symbol]
            df_temp.index=df_temp['date']
            df_temp = df_temp[['bid']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_bid_temp = pd.concat([df_bid_temp, df_temp], axis=0)
        except:
            error_bid.append(symbol)
    return df_bid_temp, error_bid

df_bid, error_bid = bid_func()
print(len(error_bid))
df_bid.head(2)

bid_func function took 95856.167 ms
104


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,118.1250,107.6875,102.0000,96.0000,103.2500,115.7500,112.3125,105.5625,105.0,106.8125,...,45.74,45.85,46.43,46.22,43.98,42.7,44.59,44.97,44.81,45.15
TROW,35.0625,34.0000,33.8125,34.9375,34.6875,35.0625,35.0000,36.4375,38.5,39.0000,...,91.03,90.76,88.37,87.30,86.77,84.9,89.66,91.36,91.34,92.34


In [59]:
pd.Series(error_bid).to_csv('error_bid.csv')

C:\PROGRA1\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


### ASKHI TABLE

In [48]:
@timing
def askhi_func():
    error_askhi = []
    df_askhi = df[['date', 'symbol', 'askhi']]
    df_askhi_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:
            df_temp = df_askhi[df_askhi['symbol']==symbol]
            df_temp.index=df_temp['date']
            df_temp = df_temp[['askhi']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_askhi_temp = pd.concat([df_askhi_temp, df_temp], axis=0)

        except:
            error_bid.append(symbol)
    return df_askhi_temp, error_askhi

df_askhi, error_askhi = askhi_func()
print(len(error_askhi))
df_askhi.head(2)

askhi_func function took 96612.122 ms
0


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,125.1875,118.625,106.3750,105.0,103.5,116.00,114.7500,112.25,109.8750,111.375,...,47.2300,48.34,47.44,46.95,46.21,43.8200,44.61,45.09,45.765,45.50
TROW,37.2500,35.125,34.9375,35.0,35.5,35.75,35.5625,36.50,39.6875,39.125,...,93.3252,92.64,92.12,88.99,90.33,87.5325,89.73,91.37,92.420,92.69


### ASK TABLE

In [49]:
@timing
def ask_func():
    error_ask = []

    df_ask = df[['date', 'symbol', 'ask']]
    ########----------------#######
    df_ask_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:            
            df_temp = df_ask[df_ask['symbol']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['ask']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_ask_temp = pd.concat([df_ask_temp, df_temp], axis=0)

        except:
            error_ask.append(symbol)
    return df_ask_temp, error_ask

df_ask, error_ask = ask_func()
print(len(error_askhi))
df_ask.head(2)

ask_func function took 97308.302 ms
0


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,118.1875,107.7500,102.0,95.875,103.375,115.875,112.375,105.625,105.125,106.8750,...,45.75,45.86,46.44,46.23,43.99,42.71,44.6,44.98,44.82,45.16
TROW,35.1250,34.0625,34.0,35.000,34.750,35.125,35.125,36.500,38.625,39.0625,...,91.09,90.77,88.40,87.32,86.78,84.95,89.7,91.37,91.35,92.35


### PRICE TABLE

In [50]:
@timing
def price_func():
    error_price = []
    
    df_price = df[['date', 'symbol', 'price']]
    ########----------------#######
    df_price_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:
            df_temp = df_price[df_price['symbol']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['price']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_price_temp = pd.concat([df_price_temp, df_temp], axis=0) ### TEMP
        except:
            error_price.append(symbol)
    return df_price_temp, error_price

df_price, error_price = price_func()
print(len(error_price))
df_price.head(2)

price_func function took 96347.708 ms
104


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,118.125,107.6875,102.0,96.0,103.375,115.750,112.375,105.625,105.0625,106.8125,...,45.73,45.85,46.45,46.24,44.00,42.69,44.59,44.97,44.82,45.15
TROW,35.125,34.0625,34.0,35.0,34.750,35.125,35.000,36.500,38.5000,39.0625,...,91.03,90.80,88.41,87.30,86.78,84.95,89.65,91.31,91.34,92.32


In [60]:
pd.Series(error_price).to_csv('error_price.csv')

C:\PROGRA1\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


### VOLUME TABLE

In [51]:
@timing
def volume_func():
    error_volume = []
    
    df_volume = df[['date', 'symbol', 'vol']]
    ########----------------#######
    df_volume_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:            
            df_temp = df_volume[df_volume['symbol']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['vol']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_volume_temp = pd.concat([df_volume_temp, df_temp], axis=0) ### TEMP    
        except:
            error_volume.append(symbol)
    return df_volume_temp, error_volume

df_volume, error_volume = volume_func()
print(len(error_volume))
df_volume.head(2)        

volume_func function took 95970.817 ms
104


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,24831819.0,29568565.0,42032160.0,27848885.0,23218034.0,23630411.0,21910101.0,21110284.0,14255122.0,14531999.0,...,30441145.0,42855600.0,34397250.0,35165179.0,58768410.0,17230381.0,21428929.0,19048702.0,39141279.0,14932651.0
TROW,381616.0,545646.0,535862.0,252966.0,237955.0,207805.0,210289.0,364494.0,886817.0,519978.0,...,1959973.0,1804473.0,1863185.0,2156700.0,4232675.0,1005519.0,1587486.0,1290839.0,1093473.0,1449392.0


In [ ]:
pd.Series()

### RETURNS TABLE

In [53]:
@timing
def returns_func():
    error_returns = []
    df_return = df[['date', 'symbol', 'ret']]
    ########----------------#######
    df_return_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:
            df_temp = df_return[df_return['symbol']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['ret']]
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_return_temp = pd.concat([df_return_temp, df_temp], axis=0) ### TEMP
        except:
            error_returns.append(symbol)
    return df_return_temp, error_returns

df_returns, error_returns = returns_func()
print(len(error_returns))
df_returns.head(2)            

returns_func function took 323667.309 ms
104


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,0.054099,-0.08836,-0.052815,-0.058824,0.076823,0.11971,-0.029158,-0.060067,-0.005325,0.016657,...,-0.01867,0.002624,0.013086,-0.004521,-0.048443,-0.029773,0.044507,0.008522,-0.003336,0.007363
TROW,-0.049069,-0.030249,-0.001835,0.029412,-0.007143,0.010791,-0.003559,0.042857,0.054795,0.01461,...,-0.010221,-0.002527,-0.026322,-0.012555,-0.005957,-0.021088,0.055327,0.018516,0.000329,0.010729


### OPEN PRICE TABLE

In [54]:
@timing
def open_func():
    error_open = []
    df_open = df[['date', 'symbol', 'open']]
    ########----------------#######
    df_open_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:            
            df_temp = df_open[df_open['symbol']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['open']] 
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_open_temp = pd.concat([df_open_temp, df_temp], axis=0) ### TEM
        except:
            error_open.append(symbol)
    return df_open_temp, error_returns

df_open, error_open = open_func()
print(len(error_open))
df_open.head(2)                    

open_func function took 99312.981 ms
104


date,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-10,2000-01-11,2000-01-12,2000-01-13,2000-01-14,...,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-24,2018-12-26,2018-12-27,2018-12-28,2018-12-31
ORCL,124.6250,115.5000,101.625,100.15625,95.00000,108.0000,112.625,112.2500,108.500,109.00,...,46.42,48.10,46.06,46.05,46.08,43.77,43.0,43.95,45.14,45.19
TROW,37.0625,34.6875,33.750,33.93750,35.21875,35.3125,35.000,35.5625,36.625,38.75,...,91.96,91.81,91.22,87.72,87.88,86.22,85.5,88.02,91.79,92.08


### SHARES OUTSTANDING TABLE

In [55]:
@timing
def shares_out_func():
    error_shares_out = []
    df_shares_out = df[['date', 'symbol', 'sout']]
    ########----------------#######
    df_shares_out_temp = pd.DataFrame([])
    for symbol in list_symbol:
        try:
            df_temp = df_shares_out[df_shares_out['symbol']==symbol] ### NO _temp IN THE FILTER
            df_temp.index=df_temp['date']
            df_temp = df_temp[['s out']] 
            df_temp = df_temp.T
            df_temp.index = [symbol]
            #df_temp.rename()
            df_shares_out_temp = pd.concat([df_shares_out_temp, df_temp], axis=0) ### TEMP    
        except:
            error_shares_out.append(symbol)
    return df_shares_out_temp, error_shares_out

df_shares_out, error_shares_out = shares_out_func()
print(len(error_shares_out))
df_shares_out.head(2)                  

shares_out_func function took 88111.899 ms
641


""
